In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.corpus import stopwords
import pickle
import json
# nltk.download("stopwords")
sns.set_style('darkgrid')

In [2]:
loaded_cosine_similarities = pickle.load(open("./pickle_files/model.pkl", "rb"))
loaded_df_percent = pickle.load(open("./pickle_files/model_data_set.pkl", "rb"))
loaded_indices = pickle.load(open("./pickle_files/model_indices.pkl", "rb"))

In [3]:
def recommend(name, cosine_similarities = loaded_cosine_similarities):
    recommend_restaurant = []
    print(name)
    idx = loaded_indices[loaded_indices == name].index[0]
#     print("index: ",idx)
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
#     print(score_series)
    top30_indexes = list(score_series.iloc[0:31].index)
#     print(top30_indexes)
    for each in top30_indexes:
        recommend_restaurant.append(list(loaded_df_percent.index)[each])
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(loaded_df_percent[['cuisines','Mean Rating', 'cost', "location", "dish_liked", "address"]][loaded_df_percent.index == each].sample()))
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep="first")
#     df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
#     df_new = df_new.head(10)
#     print(df_new)
#     print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' ,str(len(df_new)), name)
    return df_new

In [4]:
# recommended = recommend("Pai Vihar")

In [5]:
all_locations = sorted(list(set(loaded_indices.values)))
# loaded_indices[loaded_indices == "Truffles"].index[0]
loaded_df_percent.sort_values(by="Mean Rating", ascending=False).drop_duplicates(inplace=False, keep="first").head(20)

,url,address,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,listed_in(type),listed_in(city),Mean Rating,review_list,overall_review
name,,,,,,,,,,,,,,,,,
SantÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Spa Cuisine,https://www.zomato.com/bangalore/sant%C3%A9-sp...,"151, 2nd Cross, Domlur 2nd Stage, Indiranagar,...",No,Yes,4.9,246,Indiranagar,Casual Dining,"Biryani, Tomato Soup, Farm Pizza, Dumplings, L...","Healthy Food, Salad, Mediterranean","1,000",rated 50 ratedn visited lovely place recently ...,Dine-out,Old Airport Road,5.00,"[('rated 5.0', ""rated\n i visited this lovely...","Biryani, Tomato Soup, Farm Pizza, Dumplings, L..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2217,Koramangala 5th Block,"Casual Dining, Bar","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns...","Asian, Chinese, Thai, Momos","1,500",rated 50 ratedn everything appetizers dessert ...,Delivery,Koramangala 4th Block,5.00,"[('rated 5.0', ""rated\n everything from the a...","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2227,Koramangala 5th Block,"Casual Dining, Bar","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns...","Asian, Chinese, Thai, Momos","1,500",rated 50 ratedn tempura shrimp sushi rolls 55n...,Dine-out,Koramangala 4th Block,5.00,"[('rated 5.0', 'rated\n tempura shrimp sushi ...","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2244,Koramangala 5th Block,"Casual Dining, Bar","Noodles, Chicken Dim Sum, Pad Thai Noodle, Jum...","Asian, Chinese, Thai, Momos","1,500",rated 30 ratedn hmm great bad buffet ok spread...,Delivery,Koramangala 6th Block,5.00,"[('rated 3.0', 'rated\n hmm not great not bad...","Noodles, Chicken Dim Sum, Pad Thai Noodle, Jum..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2202,Koramangala 5th Block,"Casual Dining, Bar","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns...","Asian, Chinese, Thai, Momos","1,500",rated 50 ratedn everything appetizers dessert ...,Delivery,Jayanagar,5.00,"[('rated 5.0', ""rated\n everything from the a...","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2215,Koramangala 5th Block,"Casual Dining, Bar","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns...","Asian, Chinese, Thai, Momos","1,500",rated 50 ratedn everything appetizers dessert ...,Buffet,Koramangala 4th Block,5.00,"[('rated 5.0', ""rated\n everything from the a...","Chicken Dim Sum, Pad Thai Noodle, Jumbo Prawns..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2249,Koramangala 5th Block,"Casual Dining, Bar","Noodles, Chicken Dim Sum, Pad Thai Noodle, Jum...","Asian, Chinese, Thai, Momos","1,500",rated 30 ratedn hmm great bad buffet ok spread...,Dine-out,Koramangala 6th Block,5.00,"[('rated 3.0', 'rated\n hmm not great not bad...","Noodles, Chicken Dim Sum, Pad Thai Noodle, Jum..."
Asia Kitchen By Mainland China,https://www.zomato.com/bangalore/asia-kitchen-...,"136, Ground Floor, 1st Cross, 5th Block, Jyoti...",Yes,Yes,4.9,2249,Koramangala 5th Block,"Casual Dining, Bar","Noodles, Chicken Dim Sum, Pad Thai Noodle, Jum...","Asian, Chinese, Thai, Momos","1,500",rated 30 ratedn hmm great bad buffet ok spread...,Buffet,Koramangala 7th Block,5.00,"[('rated 3.0', 'rated\n hmm not great not bad...","Noodles, Chicken Dim Sum, Pad Thai Noodle, Jum..."
Asia 

In [6]:
def getBasedOnLocation(location):
    all_rest_location = loaded_df_percent[loaded_df_percent["location"]==location]
    all_rest_location.reset_index(inplace=True)
    all_rest_location.drop_duplicates(keep="first", subset="name", inplace=True)
    all_rest_location.sort_values(by="Mean Rating", ascending=False)
    restaurant_list = []
#     df_new = pd.DataFrame(columns=['cuisines','Mean Rating', 'cost', "location", "dish_liked", "address"])
#     for each in all_rest_location:
#         df_new = df_new.append(pd.DataFrame(loaded_df_percent[['cuisines','Mean Rating', 'cost', "location", "dish_liked", "address"]][loaded_df_percent.index == each]))
    
    for rest in all_rest_location.index.values[:31]:
        res = {}
        res["name"] = str(all_rest_location.at[rest, "name"])
        res["location"] = str(all_rest_location.at[rest,'location'])
        res["cuisines"] = list([cu.strip() for cu in str(all_rest_location.at[rest,"cuisines"]).split(",")])
#         try: 
            
#         except:
# #             print("PROBLEM: ",all_rest_location.at[rest,"cuisines"], type(all_rest_location.at[rest,"cuisines"]))
#             res["cuisines"] = all_rest_location.at[rest,"cuisines"]
        res["Mean Rating"] = str(all_rest_location.at[rest,"Mean Rating"])
        
        res["cost"] = str(all_rest_location.at[rest,"cost"])

        res["dish_liked"] = list([dish.strip() for dish in str(all_rest_location.at[rest,"dish_liked"]).split(", ")])

        res["address"] = str(all_rest_location.at[rest,"address"])
#         print(res)
        restaurant_list.append(res)
    print(type(restaurant_list))
    return restaurant_list
    

In [7]:
# all_rest_location = loaded_df_percent[loaded_df_percent["location"]=="Malleshwaram"]
# all_rest_location.reset_index(inplace=True)
# all_rest_location.drop_duplicates(keep="first", subset="name", inplace=True)
# all_rest_location
# restaurant_list = []
# for rest in all_rest_location.index.values:
#     res = {}
# #         print(rest)
#     res["name"] = rest
#     res["location"] = all_rest_location.at[rest,'location']
# #     print("location: ***",res["location"])
#     try: 
#         res["cuisines"] = [cu.strip() for cu in str(all_rest_location.at[rest,"cuisines"]).split(",")]
#     except:
# #             print("PROBLEM: ",all_rest_location.at[rest,"cuisines"], type(all_rest_location.at[rest,"cuisines"]))
#         res["cuisines"] = all_rest_location.at[rest,"cuisines"]
#     res["Mean Rating"] = all_rest_location.at[rest,"Mean Rating"]
# #     print("Rating: ***",res["Mean Rating"])
#     res["cost"] = all_rest_location.at[rest,"cost"]
# #     print("cost: ***",res["cost"])
#     res["dish_liked"] = [dish.strip() for dish in str(all_rest_location.at[rest,"dish_liked"]).split(", ")]
# #     print("dishes: ***",res["dish_liked"])
#     res["address"] = all_rest_location.at[rest,"address"]
# #     print("address: ***",res["address"])
# #         print(res)
#     restaurant_list.append(res)
# print(restaurant_list)


In [ ]:
from flask import Flask, jsonify, request 
from flask_cors import CORS
app = Flask(__name__)

CORS(app)

@app.route("/ping", methods=["GET"])
def home():
    if(request.method == "GET"):
        data = "pong"
        return jsonify({"data": data})

@app.route("/get-locations", methods=["GET"])
def location():
    if(request.method == "GET"):
        data = all_locations
        return jsonify({"data": data})
    
@app.route("/recommend-me", methods=["GET"])
def recommendPlaces():
    if(request.method == "GET"):
        recommended_reviews = recommend(request.args.get("restname"))
#         recommended_cu = recommend(request.args.get(""))
        restaurant_list = []
        print(recommended_reviews)
        for rest in recommended_reviews.index.values:
            res = {}
            print(rest)
            res["name"] = str(rest)
            res["location"] = str(recommended_reviews.at[rest,'location'])
            res["cuisines"] = list([cu.strip() for cu in str(recommended_reviews.at[rest,"cuisines"]).split(", ")])
            res["Mean Rating"] = str(recommended_reviews.at[rest,"Mean Rating"])
            res["cost"] = str(recommended_reviews.at[rest,"cost"])
            res["dish_liked"] = list([dish.strip() for dish in str(recommended_reviews.at[rest,"dish_liked"]).split(", ")])
            res["address"] = str(recommended_reviews.at[rest,"address"])
            restaurant_list.append(res)
        return jsonify({"data": list(restaurant_list)})

@app.route("/restaurant-on-location", methods=["GET"])
def getLocationPlaces():
    if(request.method=="GET"):
        recommended_location = getBasedOnLocation(request.args.get("location"))
        return jsonify({"data": recommended_location})

@app.route("/get-areas", methods=["GET"])
def getAreas():
    if(request.method == "GET"):
        return jsonify({"data": list(loaded_df_percent["location"].unique())})
    
if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("localhost", 9000, app)
#     app.run(debug = True)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jul/2022 13:52:57] "GET /get-locations HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 13:52:58] "GET /get-areas HTTP/1.1" 200 -


Matru Sagar


127.0.0.1 - - [20/Jul/2022 13:53:34] "GET /recommend-me?restname=Matru%20Sagar HTTP/1.1" 200 -


                                                                    cuisines  \
Matru Sagar                              South Indian, North Indian, Chinese   
Matru Sagar                              South Indian, North Indian, Chinese   
CTR                                                             South Indian   
Chetty's Coffee                                                 South Indian   
Podi Idly                                                       South Indian   
Sagar Samrat                             North Indian, Chinese, South Indian   
Palmgrove                        North Indian, South Indian, Chinese, Juices   
Southinn                   South Indian, Fast Food, Chinese, North Indian...   
A2B - Adyar Ananda Bhavan   South Indian, North Indian, Chinese, Street Food   
Sri Udupi Park                South Indian, North Indian, Chinese, Beverages   
Sri Ayodhya Veg                                   South Indian, North Indian   
Hotel Janatha                           

127.0.0.1 - - [20/Jul/2022 13:55:41] "GET /recommend-me?restname=Sandwich%20Truffles HTTP/1.1" 200 -


...

Tewari Bros Mithai Shoppie
56 Bhoga Sweets


127.0.0.1 - - [20/Jul/2022 16:15:52] "GET /get-locations HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 16:15:52] "GET /get-areas HTTP/1.1" 200 -


Truffles


127.0.0.1 - - [20/Jul/2022 16:16:03] "GET /recommend-me?restname=Truffles HTTP/1.1" 200 -


                                                                   cuisines  \
Truffles                                      Cafe, American, Burger, Steak   
Cinnamon                                     North Indian, Chinese, Biryani   
Cafe Azzure                              Cafe, Continental, Italian, Burger   
Communiti                                           Continental, BBQ, Salad   
Wolf'ish                             Cafe, American, Italian, Pizza, Burger   
Smally's Resto Cafe                  Cafe, Italian, Burger, American, Steak   
Bundar                                    Cafe, Burger, Fast Food, Desserts   
Brooks and Bonds Brewery  Continental, Mediterranean, North Indian, Chin...   
Biergarten                       Continental, European, BBQ, Chinese, Asian   

                          Mean Rating   cost               location  \
Truffles                         4.61    900         St. Marks Road   
Cinnamon                         3.62    550                    HSR